## Iterative Singular Value Decomposition (iSVD-mat)

In [1]:
import numpy as np
from scipy.linalg import sqrtm

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def isvd(dense_mat, sparse_mat, rank, maxiter = 100):
    
    N, T = sparse_mat.shape
    ind = sparse_mat != 0
    pos_miss = np.where(sparse_mat == 0)
    pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    dense_test = dense_mat[pos_test]
    del dense_mat
    
    ## Initialization
    mu = np.mean(sparse_mat[sparse_mat != 0])
    bias_row = np.zeros(N)
    bias_col = np.zeros(T)
    temp = sparse_mat - mu
    for n in range(N):
        bias_row[n] = np.mean(temp[n, :][sparse_mat[n, :] != 0])
    for t in range(T):
        bias_col[t] = np.mean(temp[:, t][sparse_mat[:, t] != 0])
    mat = sparse_mat.copy()
    del sparse_mat
    mat[pos_miss] = (mu + bias_row.reshape([N, 1]) + bias_col.reshape([1, T]))[pos_miss]
    
    ## Iterative SVD
    show_iter = 10
    for it in range(maxiter):
        u, s, v = np.linalg.svd(mat, full_matrices = False)
        mat_hat = u[:, : rank] @ np.diag(s[: rank]) @ v[: rank, :]
        mat[pos_miss] = mat_hat[pos_miss]
        if (it + 1) % show_iter == 0:
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, mat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, mat[pos_test])))
            print()
        
    return mat

### Evaluation on Guangzhou Speed Data

In [2]:
import scipy.io

tensor = scipy.io.loadmat('datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

In [3]:
import time

start = time.time()
rank = 10
mat_hat = isvd(dense_mat, sparse_mat, rank)
end = time.time()
print('Running time: %d seconds'%(end - start))
print()

Iter: 10
MAPE: 0.104059
RMSE: 4.41633

Iter: 20
MAPE: 0.104076
RMSE: 4.43407

Iter: 30
MAPE: 0.104119
RMSE: 4.44041

Iter: 40
MAPE: 0.104144
RMSE: 4.44323

Iter: 50
MAPE: 0.104157
RMSE: 4.44455

Iter: 60
MAPE: 0.104164
RMSE: 4.44518

Iter: 70
MAPE: 0.104168
RMSE: 4.44549

Iter: 80
MAPE: 0.104171
RMSE: 4.44563

Iter: 90
MAPE: 0.104172
RMSE: 4.4457

Iter: 100
MAPE: 0.104174
RMSE: 4.44573

Running time: 15 seconds

